In [ ]:
import os
import re
import numpy as np
import inflect

path = "/home/sarehalli/Code/PTB/tagged/pos/wsj/"

data = []
for x in os.listdir(path):
    for y in os.listdir(path+x):
        with open(path+x+"/"+y) as in_f:
            for line in in_f.readlines():
                for token in line.split():
                    if "/" not in token:
                        continue
                    try:
                        word, tag = re.split(r'(?<!\\)/', token)
                        data.append((word, tag))
                    except:
                        print(token)

data[:10]

In [ ]:
pl = inflect.engine()

In [ ]:
words = list(set([w for w,_ in data]))
tags = list(set([t for _,ts in data for t in ts.split("|")]))
tag_by_word = {w:{t:0 for t in tags} for w in words}

for word,tag in data:
    tag_by_word[word.lower()] = {x:((y+(1/len(tag.split("|")))) if (x in tag.split("|")) else y) 
                                 for x,y in tag_by_word.get(word, {}).items()}



In [ ]:
# verify that we have partial counts for ambiguity

for w,ts in tag_by_word.items():
    for t,c in ts.items():
        if c % 1 != 0:
            print(w,t,c)

In [ ]:
infverb_tags = ["VBP", "VBZ"]

tag_by_infverb = {x:y for x,y in tag_by_word.items() if sum([(y.get(z, 0) > 0) for z in infverb_tags]) > 0}

for word,tag_counts in tag_by_infverb.items():
    small_tag_counts = {}
    for tag, count in tag_counts.items():
        if tag in infverb_tags:
            small_tag_counts[tag] = small_tag_counts.get(tag, 0) + count
        else:
            small_tag_counts["other"] = small_tag_counts.get("other", 0) + count
    tag_by_infverb[word] = small_tag_counts
    

In [ ]:
tagdist_by_verb = {w:{t:c/sum(ts.values()) for t,c in ts.items()} for w,ts in tag_by_infverb.items()}
tagdist_by_verb 

In [ ]:
balanced_tdbv = {}

word_pairs = []
for word,ts in tagdist_by_verb.items():
    if ts["VBZ"] > 0:
        try:
            plural = pl.plural(word)
            if plural in tagdist_by_verb:
                balanced_tdbv[word] = ts
                balanced_tdbv[plural] = tagdist_by_verb[plural]
                word_pairs.append((word, plural))
        except: 
            print(word)
            
len(word_pairs)

In [ ]:
import pickle

with open("verb_pairs.dict", "wb") as out_f:
    pickle.dump(word_pairs, out_f)

with open("verb_dist.dict", "wb") as out_f:
    pickle.dump(balanced_tdbv, out_f)

In [ ]:
with open("verb_dist.dict", "rb") as in_f:
    tdbv = pickle.load(in_f)
    
tdbv

In [ ]:
import csv
# convert to dictwriter format
dw_out = []

for w,ts in tagdist_by_verb.items():
    ts["word"] = w
    dw_out.append(ts)
    
with open("verb_dist.csv", "w") as out_f:
    dw = csv.DictWriter(out_f, fieldnames=dw_out[0].keys())
    dw.writeheader()
    dw.writerows(dw_out)

# Run models using eval_ccgmulti with paradigm allverbs_prod

Then move to the R notebook!